In [1]:
from py5paisa import FivePaisaClient
from py5paisa.order import Order, OrderType, Exchange, Bo_co_order
import time
import math
import pandas as pd
import warnings
from datetime import datetime

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
cred = {
    "APP_NAME": "5P56409084",
    "APP_SOURCE": "9217",
    "USER_ID": "pybeQPEbjju",
    "PASSWORD": "AESDZQGqyAa",
    "USER_KEY": "542GTEZguBbLERUXRZe26rLvfWU2X2KD",
    "ENCRYPTION_KEY": "l8sRTIsjl1JhUrz8X48XQJzKGI4k0gft",
}
client = FivePaisaClient(
    email="vasuappliedai@gmail.com", passwd="Vasudeva@1", dob="19981218", cred=cred
)
client.login()

 19:38:01 | Logged in!!


In [11]:
def adjust_strangle(option_type,desired_price,entry_point):
    call_df, put_df = get_current_expiry_strikes(bank_nifty_symbol, strikePrice*0.94, strikePrice*1.06)
    if option_type == "CE":
        call_option_chain = map_prices_to_strike_prices(call_df,entry_point)
        call_script_code = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["Scripcode"].iloc[0]
        call_sell = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["LTP"].iloc[0]
        call_symbol = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["Name"].iloc[0]
        print("Sold call call_sell, call_symbol, call_script_code ",call_sell,call_symbol, call_script_code)
    if option_type == "PE":
        put_option_chain = map_prices_to_strike_prices(put_df,entry_point)
        put_script_code = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)["Scripcode"].iloc[0]
        put_sell = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)["LTP"].iloc[0]
        put_symbol = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)["Name"].iloc[0]
        print("Sold put sell put_sell, put_symbol, put_script_code ",put_sell,put_symbol,put_script_code)

def get_current_expiry_strikes(symbol, start_strike, end_strike):
    temp_df = script_df["FullName"].str.contains(
        pat=symbol + " CE", regex=True)
    call_df = script_df[temp_df]
    call_df = call_df[call_df["StrikeRate"] > start_strike]
    call_df = call_df[call_df["StrikeRate"] < end_strike]
    call_df.to_csv("bank_nifty_call_expiry.csv")
    temp_df = script_df["FullName"].str.contains(
        pat=symbol + " PE", regex=True)
    put_df = script_df[temp_df]
    put_df = put_df[put_df["StrikeRate"] > start_strike]
    put_df = put_df[put_df["StrikeRate"] < end_strike]
    put_df.to_csv("bank_nifty_put_expiry.csv")
    return call_df, put_df

def map_prices_to_strike_prices(data_frame, index_point=None):
    temp_rows = []
    for _, row in data_frame.iterrows():
        script_code = row["Scripcode"]
        try:
            if index_point is None:
                close_point = client.historical_data("N", "D", script_code, "5m", 
                    given_date, given_date).tail(1)
                temp_rows.append(close_point["Close"].iloc[0])
            else:
                close_point = client.historical_data("N", "D", script_code, "1m", 
                    given_date, given_date).loc[index_point]
                temp_rows.append(close_point["Close"])
        except:
            temp_rows.append(9999)
            continue
    data_frame["LTP"] = temp_rows
    return data_frame.sort_values(by=["LTP"])


        
def rounded_number(number):
    return (round(strikePrice / 1000, 1) * 1000).astype(int)

In [5]:
# This code should be executed on Thursday or when profit is booked
nifty_script_code = 999920000
banknifty_script_code = 999920005
given_date = datetime.today().strftime("%Y-%m-%d")
script_df = pd.read_csv("scripmaster-csv-format.csv",low_memory=False)
bank_nifty_symbol = "BANKNIFTY 31 Mar 2022"
strikePrice = client.historical_data("N", "C", banknifty_script_code, "1m", given_date, given_date).tail(1)["Open"].iloc[0]
strikePrice = rounded_number(strikePrice)
strikePrice

35400

In [ ]:
call_df, put_df = get_current_expiry_strikes(bank_nifty_symbol, strikePrice*0.94, strikePrice*1.06)
call_df,put_df

In [7]:
call_option_chain = map_prices_to_strike_prices(call_df)
put_option_chain = map_prices_to_strike_prices(put_df)

In [ ]:
import pickle
# option_chains = (call_option_chain,put_option_chain)
# with open("option_chain.pickle","wb") as file:
#     pickle.dump(option_chains,file)

file = open("option_chain.pickle","rb")
call_option_chain,put_option_chain = pickle.load(file)

In [ ]:
desired_call_price = 115
desired_put_price = 115

In [ ]:
call_option_chain[call_option_chain["LTP"]<desired_call_price].tail(1)

In [ ]:
put_option_chain[put_option_chain["LTP"]<desired_put_price].tail(1)

In [ ]:
call_script_code = call_option_chain[call_option_chain["LTP"]<desired_call_price].tail(1)["Scripcode"].iloc[0]
put_script_code = put_option_chain[put_option_chain["LTP"]<desired_put_price].tail(1)["Scripcode"].iloc[0]
call_script_code,put_script_code

In [24]:
call_script_code = 58304
put_script_code = 58211


In [16]:
def start_strangle(entry_point_index,call_script_code,put_script_code):
    call_points = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)["Close"].to_list()[entry_point_index:]
    put_points = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)["Close"].to_list()[entry_point_index:]
    for i in range(min(len(call_points),len(put_points))):
        call_point = call_points[i]
        put_point = put_points[i]
        if(call_point*0.5 > put_point):
            print("PE is half of CE at index, call_price and put_price ",entry_point_index+i,call_point,put_point)
            entry_point_index = i
            adjust_strangle("PE",call_point*0.8,entry_point_index)
            return "PE",entry_point_index
        elif(call_point < put_point*0.5):        
            print("CE is half of PE at index, call_price and put_price ",entry_point_index+i,call_point,put_point)
            entry_point_index = i
            adjust_strangle("CE",put_point*0.8,entry_point_index)
            return "CE",entry_point_index

start_strangle(273,call_script_code,put_script_code)

In [21]:
def day_close_check(call_script_code,put_script_code):
    call_point = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)["Close"].iloc[370]
    put_point = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)["Close"].iloc[370]
    if(call_point < put_point*0.75):
        print("Buy CE at ", call_point)
        print("Buy CE and sell CE near to put strike price")
        adjust_strangle("CE",put_point*0.8,370)
    if(put_point < call_point*0.75):
        print("Buy PE at ", put_point)
        print("selling PE near to call strike price")
        adjust_strangle("PE",call_point*0.8,370)
    
day_close_check(call_script_code,put_script_code)

Buy PE at  74.2
selling PE near to call strike price
Sold put sell put_sell, put_symbol, put_script_code  83.6 BANKNIFTY 31 Mar 2022 PE 33900.00 58211


In [25]:
call_point = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)["Close"].iloc[374]
put_point = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)["Close"].iloc[374]
call_point,put_point 

(101.95, 80.0)